In [9]:
from datetime import datetime

from google.cloud import aiplatform

In [1]:
# variables

REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


# train

In [2]:
%%writefile train.py
"""Covertype Classifier trainer script."""
import os
import pickle
import subprocess
import sys

import fire
import hypertune
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

AIP_MODEL_DIR = os.environ["AIP_MODEL_DIR"]
MODEL_FILENAME = "model.pkl"


def train_evaluate(
    training_dataset_path, validation_dataset_path, alpha, max_iter
):
    """Trains the Covertype Classifier model."""
    df_train = pd.read_csv(training_dataset_path)
    df_validation = pd.read_csv(validation_dataset_path)

    df_train = pd.concat([df_train, df_validation])

    numeric_features = [
        "Elevation",
        "Aspect",
        "Slope",
        "Horizontal_Distance_To_Hydrology",
        "Vertical_Distance_To_Hydrology",
        "Horizontal_Distance_To_Roadways",
        "Hillshade_9am",
        "Hillshade_Noon",
        "Hillshade_3pm",
        "Horizontal_Distance_To_Fire_Points",
    ]

    categorical_features = ["Wilderness_Area", "Soil_Type"]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("cat", OneHotEncoder(), categorical_features),
        ]
    )

    pipeline = Pipeline(
        [
            ("preprocessor", preprocessor),
            ("classifier", SGDClassifier(loss="log")),
        ]
    )

    num_features_type_map = {feature: "float64" for feature in numeric_features}
    df_train = df_train.astype(num_features_type_map)
    df_validation = df_validation.astype(num_features_type_map)

    print(f"Starting training: alpha={alpha}, max_iter={max_iter}")
    # pylint: disable-next=invalid-name
    X_train = df_train.drop("Cover_Type", axis=1)
    y_train = df_train["Cover_Type"]

    pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
    pipeline.fit(X_train, y_train)


    # Save the model
    with open(MODEL_FILENAME, "wb") as model_file:
        pickle.dump(pipeline, model_file)
    subprocess.check_call(
        ["gsutil", "cp", MODEL_FILENAME, AIP_MODEL_DIR], stderr=sys.stdout
    )
    print(f"Saved model in: {AIP_MODEL_DIR}")


if __name__ == "__main__":
    fire.Fire(train_evaluate)

Writing train.py


In [3]:
%%writefile Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing Dockerfile


In [4]:
IMAGE_NAME = "trainer_image_covertype_vertex"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"
TRAINING_CONTAINER_IMAGE_URI

'gcr.io/qwiklabs-gcp-04-853e5675f5e8/trainer_image_covertype_vertex:latest'

In [5]:
# !gcloud builds submit --async --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI folder
!gcloud builds submit --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI .

Creating temporary tarball archive of 4 file(s) totalling 92.4 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-gcp-04-853e5675f5e8_cloudbuild/source/1647340872.262519-8a57a5222203492cb7bd98fb477c71df.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-04-853e5675f5e8/locations/global/builds/c2776090-1640-4fc4-bddb-cc85c9987260].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/c2776090-1640-4fc4-bddb-cc85c9987260?project=1076138843678].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "c2776090-1640-4fc4-bddb-cc85c9987260"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-04-853e5675f5e8_cloudbuild/source/1647340872.262519-8a57a5222203492cb7bd98fb477c71df.tgz#1647340872639800
Copying gs://qwiklabs-gcp-04-853e5675f5e8_cloudbuild/source/1647340872.262519-8a57a5222203492cb7bd98fb477c71df.tgz#1647340872639800...
/ [1 files][ 20.2 KiB/ 20.2 KiB]                             

# pipeline

In [6]:
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest"
)

In [7]:
%%writefile ./pipeline_prebuilt.py
"""Kubeflow Covertype Pipeline."""

import os
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google_cloud_pipeline_components.aiplatform import (
    EndpointCreateOp,
    ModelDeployOp,
    ModelUploadOp,
)
from google_cloud_pipeline_components.experimental import (
    hyperparameter_tuning_job,
)
from google_cloud_pipeline_components.experimental.custom_job import (
    CustomTrainingJobOp,
)
from kfp.v2 import dsl

PIPELINE_ROOT = os.getenv("PIPELINE_ROOT")
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")

TRAINING_CONTAINER_IMAGE_URI = os.getenv("TRAINING_CONTAINER_IMAGE_URI")
SERVING_CONTAINER_IMAGE_URI = os.getenv("SERVING_CONTAINER_IMAGE_URI")
SERVING_MACHINE_TYPE = os.getenv("SERVING_MACHINE_TYPE", "n1-standard-16")

TRAINING_FILE_PATH = os.getenv("TRAINING_FILE_PATH")
VALIDATION_FILE_PATH = os.getenv("VALIDATION_FILE_PATH")

# MAX_TRIAL_COUNT = int(os.getenv("MAX_TRIAL_COUNT", "5"))
# PARALLEL_TRIAL_COUNT = int(os.getenv("PARALLEL_TRIAL_COUNT", "5"))
# THRESHOLD = float(os.getenv("THRESHOLD", "0.6"))

PIPELINE_NAME = os.getenv("PIPELINE_NAME", "covertype")
BASE_OUTPUT_DIR = os.getenv("BASE_OUTPUT_DIR", PIPELINE_ROOT)
MODEL_DISPLAY_NAME = os.getenv("MODEL_DISPLAY_NAME", PIPELINE_NAME)


@dsl.pipeline(
    name=f"forest-{PIPELINE_NAME}-kfp-pipeline",
    description="Kubeflow pipeline that Trains, and deploys on Vertex",
    pipeline_root=PIPELINE_ROOT,
)
def create_pipeline():

    worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-4",
                "accelerator_type": "NVIDIA_TESLA_T4",
                "accelerator_count": 1,
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": TRAINING_CONTAINER_IMAGE_URI,
                "args": [
                    f"--training_dataset_path={TRAINING_FILE_PATH}",
                    f"--validation_dataset_path={VALIDATION_FILE_PATH}",
                    "--alpha=1.0e-1",
                    "--max_iter=1"
                ],
            },
        }
    ]

    training_task = CustomTrainingJobOp(
        project=PROJECT_ID,
        location=REGION,
        display_name=f"{PIPELINE_NAME}-kfp-training-job",
        worker_pool_specs=worker_pool_specs,
        base_output_directory=BASE_OUTPUT_DIR,
    )

    model_upload_task = ModelUploadOp(
        project=PROJECT_ID,
        display_name=f"{PIPELINE_NAME}-kfp-model-upload-job",
        artifact_uri=f"{BASE_OUTPUT_DIR}/model",
        serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    )
    model_upload_task.after(training_task)

    endpoint_create_task = EndpointCreateOp(
        project=PROJECT_ID,
        display_name=f"{PIPELINE_NAME}-kfp-create-endpoint-job",
    )
    endpoint_create_task.after(model_upload_task)

    model_deploy_op = ModelDeployOp(  # pylint: disable=unused-variable
        model=model_upload_task.outputs["model"],
        endpoint=endpoint_create_task.outputs["endpoint"],
        deployed_model_display_name=MODEL_DISPLAY_NAME,
        dedicated_resources_machine_type=SERVING_MACHINE_TYPE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

Writing ./pipeline_prebuilt.py


# deploy

In [10]:
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/data"

TRAINING_FILE_PATH = f"{DATA_ROOT}/training/dataset.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/dataset.csv"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BASE_OUTPUT_DIR = f"{ARTIFACT_STORE}/models/{TIMESTAMP}"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}
%env BASE_OUTPUT_DIR={BASE_OUTPUT_DIR}

PIPELINE_JSON = "covertype_kfp_pipeline.json"

env: PIPELINE_ROOT=gs://qwiklabs-gcp-04-853e5675f5e8-kfp-artifact-store/pipeline
env: PROJECT_ID=qwiklabs-gcp-04-853e5675f5e8
env: REGION=us-central1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-gcp-04-853e5675f5e8/trainer_image_covertype_vertex:latest
env: TRAINING_FILE_PATH=gs://qwiklabs-gcp-04-853e5675f5e8-kfp-artifact-store/data/training/dataset.csv
env: VALIDATION_FILE_PATH=gs://qwiklabs-gcp-04-853e5675f5e8-kfp-artifact-store/data/validation/dataset.csv
env: BASE_OUTPUT_DIR=gs://qwiklabs-gcp-04-853e5675f5e8-kfp-artifact-store/models/20220315104429


In [13]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-gcp-04-853e5675f5e8-kfp-artifact-store/


In [14]:
!dsl-compile-v2 --py pipeline_prebuilt.py --output $PIPELINE_JSON

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [ ]:
!head {PIPELINE_JSON}

In [41]:
aiplatform.init(project=PROJECT_ID, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="[boilerplate]covertype_kfp_pipeline",
    template_path=PIPELINE_JSON,
    enable_caching=True,
)

pipeline.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/1076138843678/locations/us-central1/pipelineJobs/forest-covertype-kfp-pipeline-20220314124519
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/1076138843678/locations/us-central1/pipelineJobs/forest-covertype-kfp-pipeline-20220314124519')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/forest-covertype-kfp-pipeline-20220314124519?project=1076138843678
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/1076138843678/locations/us-central1/pipelineJobs/forest-covertype-kfp-pipeline-20220314124519 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob pr

# Test

In [ ]:
# endpoint = 
# endpoint.predict([instance])

In [47]:
from typing import Dict, List, Union
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if type(instances) == list else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    print(parameters)
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))
        
        
        
instance = [
    2841.0,
    45.0,
    0.0,
    644.0,
    282.0,
    1376.0,
    218.0,
    237.0,
    156.0,
    1003.0,
    "Commanche",
    "C4758",
]
"""
[
    { "Elevation": "2841.0", "Aspect":"45.0", "Slope":"0.0", "Horizontal_Distance_To_Hydrology":"644.0", "Vertical_Distance_To_Hydrology":"282.0", "Horizontal_Distance_To_Roadways":"1376.0","Hillshade_9am":"218.0", "Hillshade_Noon":"237.0",  "Hillshade_3pm":"156.0", "Horizontal_Distance_To_Fire_Points":"1003.0", "Wilderness_Area":"Commanche", "Soil_Type":"C4758"}
]
"""

predict_custom_trained_model_sample(
    project="qwiklabs-gcp-04-853e5675f5e8",
    endpoint_id="5729889343875579904",
    location="us-central1",
    instances=[instance]
)

struct_value {
}



FailedPrecondition: 400 "Prediction failed: Exception during sklearn prediction: Specifying the columns using strings is only supported for pandas DataFrames"
